In [1]:
import pandas as pd
import warnings
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)
warnings.simplefilter(action='ignore', category=FutureWarning)

import numpy as np
import matplotlib.pyplot as plt
import os
import tsfresh
from datetime import timedelta

from sklearn.feature_selection import mutual_info_classif
from sklearn.preprocessing import LabelEncoder
%matplotlib inline

In [2]:
sensor_types = ['mic', 'acc']

In [3]:
def trial_sample_extraction(path, disable_contact = False):
    sensors_dict = {}
    if disable_contact:
        sensors = ['acc', 'mic']
    else:
        sensors = ['contact', 'acc', 'mic']
    for sensor in sensors:
        full_path = path + '_' + sensor + '.csv'
        if not os.path.isfile(full_path):
            return False
        sample = pd.read_csv(full_path)
        sample['time_s'] = sample['time_s'].apply(lambda epoch: epoch * 1e9)
        sample['time_s'] = pd.to_datetime(sample['time_s'])
        sample = sample.drop(sample.columns[0], axis=1)
        sample.index = sample.time_s
        sample = sample.drop(['time_s'], axis=1)
        sensors_dict[sensor] = sample
    return sensors_dict

In [4]:
def calc_jerk(acc_df):
    date = pd.Series(acc_df.index)
    date.index = pd.to_datetime(date)
    time_d = (date - date.shift())
    time_d = time_d.apply(lambda x: (x.microseconds/1000))

    acc_df['jerk_x'] = acc_df['ax'].rolling(2).apply(lambda x: (x.iloc[1] - x.iloc[0]))
    acc_df['jerk_x'] = acc_df['jerk_x'].div(time_d)

    acc_df['jerk_y'] = acc_df['ay'].rolling(2).apply(lambda x: (x.iloc[1] - x.iloc[0]))
    acc_df['jerk_y'] = acc_df['jerk_y'].div(time_d)

    acc_df['jerk_z'] = acc_df['az'].rolling(2).apply(lambda x: (x.iloc[1] - x.iloc[0]))
    acc_df['jerk_z'] = acc_df['jerk_z'].div(time_d)
    acc_df.fillna(0, inplace=True)

    return acc_df

In [5]:
def calc_stats(df):
    stats_pd = pd.DataFrame()
    for sensor_type in sensor_types:
        for column in df[sensor_type].columns:
            stats_pd['mean_' + column] = [df[sensor_type][column].mean()]
            stats_pd['std_' + column] = [df[sensor_type][column].std()]
            stats_pd['min_' + column] = [df[sensor_type][column].min()]
            stats_pd['max_' + column] = [df[sensor_type][column].max()]
            stats_pd['var_' + column] = [df[sensor_type][column].var()]
            stats_pd['kurt_' + column] = [df[sensor_type][column].kurt()]
            stats_pd['skew_' + column] = [df[sensor_type][column].skew()]
            stats_pd['median_' + column] = [df[sensor_type][column].median()]
            stats_pd['abs_energy_' + column] = [tsfresh.feature_extraction.feature_calculators.abs_energy(df[sensor_type][column])]
            stats_pd['peaks_' + column] = [tsfresh.feature_extraction.feature_calculators.number_cwt_peaks(df[sensor_type][column], 2)]
            stats_pd['derv_central_' + column] = [tsfresh.feature_extraction.feature_calculators.mean_second_derivative_central(df[sensor_type][column])]
            stats_pd['mean_abs_change_' + column] = [tsfresh.feature_extraction.feature_calculators.mean_abs_change(df[sensor_type][column])]
            stats_pd['abs_sum_change_' + column] = [tsfresh.feature_extraction.feature_calculators.absolute_sum_of_changes(df[sensor_type][column])]
            stats_pd['fourier_entropy_' + column] = [tsfresh.feature_extraction.feature_calculators.fourier_entropy(df[sensor_type][column], 2)]
            rms = np.sqrt(np.mean(df[sensor_type][column]**2))
            stats_pd['root_mean_s_' + column] = [rms]
            stats_pd['shape_factor_' + column] = [rms / (df[sensor_type][column].apply(abs).mean())]
    stats_pd.reset_index(inplace=True, drop=True)
    return stats_pd

In [6]:
def encodeLabels(df, binary, binary_target=None):
    if binary:
        df['label'] = np.where(df['label'].str.contains(binary_target), 1, 0)
    else:
        encoder = LabelEncoder()
        df['label'] = encoder.fit_transform(df['label'])

In [7]:
def discrete_entropy(y):
    bincount = np.bincount(y)
    probabilities = bincount/float(bincount.sum())
    return -(probabilities * np.log(probabilities)).sum()

def compute_rmi(X, y):
    mi_ = mutual_info_classif(X, y, discrete_features=False, random_state=101)
    e = discrete_entropy(y)
    rmi = [i/e for i in mi_]
    #rmi_ = mi_ / float(discrete_entropy(y.astype(int)))
    return rmi

In [8]:
def table_cell_format(v):
    import matplotlib.colors as colors
    import matplotlib.cm as cmx
    cmp = cm = plt.get_cmap('Greys')
    cNorm = colors.Normalize(vmin=0, vmax=2)
    scalarMap = cmx.ScalarMappable(norm=cNorm, cmap=cmp)
    color_val = scalarMap.to_rgba(v)
    r, g, b, a = color_val
    return "\cellcolor[rgb]{%.2f, %.2f, %.2f} " % (r, g, b) + "%.2f" % (v*100)

In [9]:
set_sensors = ['O1', 'O2', 'O3', 'O4', 'O5', 'O6', 'O7', 'O8']
part_names = ['participant_J', 'participant_K', 'participant_L', 'participant_E', 'participant_F', 'participant_A', 'participant_C', 'participant_G', 'participant_I',
              'participant_M', 'participant_B', 'participant_D', 'participant_H']

column_names = ["ACC", "MAG", "GYRO", "MIC"]

df_rmis = pd.DataFrame(columns=column_names)
df_coloc_rmis = pd.DataFrame(columns=column_names)

acc = ["_ax", "_ay", "_az"]
mag = ["_mx", "_my", "_mz"]
gyr = ["_gx", "_gy", "_gz"]
mic = ["_spl"]

for sensor in set_sensors:

    directory_in_str = './data_dev/' + sensor + '/day2/data/'
    directory_in_str2 = './data_dev/' + sensor + '/day1/data/'

    binary = True
    s_f_num = 20
    set_paths = set()
    set_names = set()
    first_pass = True
    first_pass_attack = True
    substrings = ['participant_F_10', 'participant_F_11', 'participant_F_12', 'participant_F_13', 'participant_F_14', 'participant_F_15']
    video_ppl = ['pE', 'pA', 'pC', 'pG', 'pI', 'pL']

    per_device_df = pd.DataFrame()
    per_device_df_attack = pd.DataFrame()

    directory = os.fsencode(directory_in_str)
    for file in os.listdir(directory):
        filename = os.fsdecode(file)
        if filename.endswith(".csv"):
            if any(x in str(filename) for x in part_names):
                path = directory_in_str + filename
                path = path[:-4]
                path_elems = path.split("_")
                str_path = ""
                str_path = str_path.join("_".join(path_elems[:-1]))
                set_paths.add(str_path)

    directory = os.fsencode(directory_in_str2)
    for file in os.listdir(directory):
        filename = os.fsdecode(file)
        if filename.endswith(".csv"):
            if any(x in str(filename) for x in part_names):
                path = directory_in_str2 + filename
                path = path[:-4]
                path_elems = path.split("_")
                str_path = ""
                str_path = str_path.join("_".join(path_elems[:-1]))
                set_paths.add(str_path)

    for path in sorted(set_paths):
        if True:

            # Extract the participant's name from the path
            path_elems_global = path.split("/")
            path_elems_local = path_elems_global[-1].split("_")
            part_name = path_elems_local[1][0] + path_elems_local[2]
            set_names.add(part_name)

            path_trial_part = "_".join(path_elems_local[1:])
            trial_stamp = path_elems_local[3] + "_" + path_elems_local[4]

            # Extract the participant's trial from the sensor folder
            sample1 = trial_sample_extraction(path)

            if len(sample1['contact']) < 2:
                continue

            time_start = sample1['contact']['status'][sample1['contact']['status'] == 'open'].index[0]
            time_stop = sample1['contact']['status'][sample1['contact']['status'] == 'close'].index[0]

            # Extract the event from the sample - open and close the doors
            for sensor_type in sensor_types:
                t1 = (sample1[sensor_type].index >= (time_start - timedelta(seconds=1)))
                t2 = (sample1[sensor_type].index <= (time_stop + timedelta(seconds=1)))
                mask = t1 & t2
                sample1[sensor_type] = sample1[sensor_type].loc[mask]

            # Calc jerk
            sample1['acc'] = calc_jerk(sample1['acc'])

            # Calc features
            sample1_stats = calc_stats(sample1)
            # sample1_stats['label'] = part_name

            first = True
            # Extract paths from other devices based on the current trial and participant

            set_sensors = ['O1', 'O2', 'O3', 'O4', 'O5', 'O6', 'O7', 'O8']

            for sens in set_sensors:
                if sens not in [sensor]:  # exclude the current device
                    if "contact" not in path_trial_part:
                        dir_ = './data_dev/' + sens + "/" + "/".join(
                            path_elems_global[3:5]) + '/' + sens + "_" + path_trial_part
                        sample_exter = trial_sample_extraction(dir_, True)
                        if sample_exter == False:
                            continue
                        for sensor_type in sensor_types:
                            t1 = (sample_exter[sensor_type].index >= (time_start - timedelta(seconds=1)))
                            t2 = (sample_exter[sensor_type].index <= (time_stop + timedelta(seconds=1)))
                            mask = t1 & t2
                            sample_exter[sensor_type] = sample_exter[sensor_type].loc[mask]

                        sample_exter['acc'] = calc_jerk(sample_exter['acc'])
                        sample_exter_stats = calc_stats(sample_exter)
                        sample_exter_stats.columns = [col + '_' + sens for col in sample_exter_stats.columns]

                        if first:
                            first = False
                            per_colo_device_df = sample_exter_stats
                        else:
                            per_colo_device_df = pd.concat([per_colo_device_df, sample_exter_stats], axis=1)

            # both main and colocated
            #helper_df = pd.concat([sample1_stats, per_colo_device_df],
            #                      axis=1)  # features from all the devices from the user's trial n authenticating interaction x

            # only colocated
            helper_df2 = per_colo_device_df

            # only main one
            helper_df = sample1_stats

            helper_df['label'] = part_name
            helper_df['trial_stamp'] = trial_stamp

            helper_df2['label'] = part_name
            helper_df2['trial_stamp'] = trial_stamp

            if 'attack' in path:
                continue

            if first_pass:
                first_pass = False
                per_device_df = pd.DataFrame(columns=helper_df.columns)
                per_device_df2 = pd.DataFrame(columns=helper_df2.columns)

            per_device_df = per_device_df.append(helper_df, ignore_index=True)
            per_device_df2 = per_device_df2.append(helper_df2, ignore_index=True)

    per_device_df = per_device_df.dropna()
    per_device_df2 = per_device_df2.dropna()

    type_ = True  # binary because we want to analyze only attacks

    if True:
        per_device_df_copy = per_device_df.copy()
        per_device_df2_copy = per_device_df2.copy()

        encodeLabels(per_device_df_copy, type_, 'pF')
        encodeLabels(per_device_df2_copy, type_, 'pF')

        X = per_device_df_copy.drop(['label', 'trial_stamp'], axis=1)
        X2 = per_device_df2_copy.drop(['label', 'trial_stamp'], axis=1)

        y = per_device_df_copy['label']
        y2 = per_device_df2_copy['label']

        # To generate a RMI table
        feats_acc = [i for e in acc for i in X if e in i]
        feats_mag = [i for e in mag for i in X if e in i]
        feats_gyr = [i for e in gyr for i in X if e in i]
        feats_mic = [i for e in mic for i in X if e in i]

        feats_acc2 = [i for e in acc for i in X2 if e in i]
        feats_mag2 = [i for e in mag for i in X2 if e in i]
        feats_gyr2 = [i for e in gyr for i in X2 if e in i]
        feats_mic2 = [i for e in mic for i in X2 if e in i]

        dev_rmi_acc = compute_rmi(X[feats_acc], y)
        acc_max = max(dev_rmi_acc)

        dev_rmi_mag = compute_rmi(X[feats_mag], y)
        mag_max = max(dev_rmi_mag)

        dev_rmi_gyr = compute_rmi(X[feats_gyr], y)
        gyr_max = max(dev_rmi_gyr)

        dev_rmi_mic = compute_rmi(X[feats_mic], y)
        mic_max = max(dev_rmi_mic)

        df_rmis.loc[sensor] = [acc_max, mag_max, gyr_max, mic_max]

        # Colocated RMI
        dev_rmi_acc = compute_rmi(X2[feats_acc2], y2)
        acc_max = max(dev_rmi_acc)

        dev_rmi_mag = compute_rmi(X2[feats_mag2], y2)
        mag_max = max(dev_rmi_mag)

        dev_rmi_gyr = compute_rmi(X2[feats_gyr2], y2)
        gyr_max = max(dev_rmi_gyr)

        dev_rmi_mic = compute_rmi(X2[feats_mic2], y2)
        mic_max = max(dev_rmi_mic)

        df_coloc_rmis.loc[sensor] = [acc_max, mag_max, gyr_max, mic_max]

print("On-device RMIs")
print(df_rmis)
print("----------------------")
print("Colocated devices RMIs")
print(df_coloc_rmis)

On-device RMIs
         ACC       MAG      GYRO       MIC
O1  0.300835  0.405805  0.718607  0.182781
O2  0.509518  0.473050  0.644498  0.253681
O3  0.629030  0.395019  0.738978  0.098231
O4  0.303868  0.332843  0.194060  0.157761
O5  0.322450  0.644622  0.481833  0.143288
O6  0.340684  0.406440  0.604763  0.416681
O7  0.372007  0.272095  0.219496  0.410017
O8  0.354303  0.541416  0.411243  0.141448
----------------------
Colocated devices RMIs
         ACC       MAG      GYRO       MIC
O1  0.792139  0.688948  0.720775  0.228050
O2  0.858592  0.537162  0.591036  0.265681
O3  0.776886  0.755310  0.742084  0.162618
O4  0.765119  0.734311  0.612789  0.233101
O5  0.792276  0.735210  0.607556  0.179314
O6  0.981899  0.694403  0.904079  0.429623
O7  0.812138  0.731229  0.575604  0.287031
O8  0.860899  0.539963  0.583710  0.206504


In [10]:
print("On-device RMIs")
print(df_rmis.to_latex(escape = False, column_format="l"+"c"*len(df_rmis.columns), formatters=[table_cell_format for i in df_rmis.columns]).replace("{} &", "Object Type &"))

On-device RMIs
\begin{tabular}{lcccc}
\toprule
Object Type &                                     ACC &                                     MAG &                                    GYRO &                                     MIC \\
\midrule
O1 & \cellcolor[rgb]{0.92, 0.92, 0.92} 30.08 & \cellcolor[rgb]{0.89, 0.89, 0.89} 40.58 & \cellcolor[rgb]{0.76, 0.76, 0.76} 71.86 & \cellcolor[rgb]{0.96, 0.96, 0.96} 18.28 \\
O2 & \cellcolor[rgb]{0.85, 0.85, 0.85} 50.95 & \cellcolor[rgb]{0.86, 0.86, 0.86} 47.30 & \cellcolor[rgb]{0.79, 0.79, 0.79} 64.45 & \cellcolor[rgb]{0.94, 0.94, 0.94} 25.37 \\
O3 & \cellcolor[rgb]{0.80, 0.80, 0.80} 62.90 & \cellcolor[rgb]{0.89, 0.89, 0.89} 39.50 & \cellcolor[rgb]{0.75, 0.75, 0.75} 73.90 &  \cellcolor[rgb]{0.98, 0.98, 0.98} 9.82 \\
O4 & \cellcolor[rgb]{0.92, 0.92, 0.92} 30.39 & \cellcolor[rgb]{0.91, 0.91, 0.91} 33.28 & \cellcolor[rgb]{0.96, 0.96, 0.96} 19.41 & \cellcolor[rgb]{0.96, 0.96, 0.96} 15.78 \\
O5 & \cellcolor[rgb]{0.92, 0.92, 0.92} 32.24 & \cellcolor[rgb]{0.

In [11]:
print("Colocated devices RMIs")
print(df_coloc_rmis.to_latex(escape = False, column_format="l"+"c"*len(df_coloc_rmis.columns), formatters=[table_cell_format for i in df_coloc_rmis.columns]).replace("{} &", "Object Type &"))

Colocated devices RMIs
\begin{tabular}{lcccc}
\toprule
Object Type &                                     ACC &                                     MAG &                                    GYRO &                                     MIC \\
\midrule
O1 & \cellcolor[rgb]{0.72, 0.72, 0.72} 79.21 & \cellcolor[rgb]{0.77, 0.77, 0.77} 68.89 & \cellcolor[rgb]{0.75, 0.75, 0.75} 72.08 & \cellcolor[rgb]{0.95, 0.95, 0.95} 22.80 \\
O2 & \cellcolor[rgb]{0.68, 0.68, 0.68} 85.86 & \cellcolor[rgb]{0.84, 0.84, 0.84} 53.72 & \cellcolor[rgb]{0.81, 0.81, 0.81} 59.10 & \cellcolor[rgb]{0.94, 0.94, 0.94} 26.57 \\
O3 & \cellcolor[rgb]{0.72, 0.72, 0.72} 77.69 & \cellcolor[rgb]{0.74, 0.74, 0.74} 75.53 & \cellcolor[rgb]{0.75, 0.75, 0.75} 74.21 & \cellcolor[rgb]{0.96, 0.96, 0.96} 16.26 \\
O4 & \cellcolor[rgb]{0.73, 0.73, 0.73} 76.51 & \cellcolor[rgb]{0.75, 0.75, 0.75} 73.43 & \cellcolor[rgb]{0.80, 0.80, 0.80} 61.28 & \cellcolor[rgb]{0.95, 0.95, 0.95} 23.31 \\
O5 & \cellcolor[rgb]{0.72, 0.72, 0.72} 79.23 & \cellcolor